In [76]:
import sys
sys.path.append('../main')
import sql
import pandas as pd
from functools import reduce
from tabulate import tabulate

In [77]:
def get(q):
    return pd.DataFrame(sql.execute(q))

In [78]:
q='''select distinct name, count(*) as total_alerts from project
    join alert a on project.id = a.project_id
    where language in ('C','C++')
    and is_invalid=0
    group by name; '''
ta=get(q)

In [79]:
q='''select name, count(*) as total_scans from project
join snapshot s on project.id = s.project_id
group by name;'''
ts=get(q)

In [80]:
q='''select name, timestampdiff(year, start_date,end_date) as "time period (years)"
from project;'''
span=get(q)

In [81]:
q='''select name, datediff(s1.date,s2.date) as intvl
        from snapshot s1
            join snapshot s2 on s1.last_snapshot=s2.id and s2.project_id=s1.project_id
        join project p on s1.project_id = p.id;'''
intvl=get(q)
intvl=intvl.groupby(intvl.name)[['intvl']].median()

In [82]:
q='select name, start_date, end_date from project'
dates= get(q)
dates.start_date=dates.start_date.dt.date
dates.end_date= dates.end_date.dt.date

In [83]:
q='''select name, count(*) as triaged from alert a
join project p on a.project_id = p.id
where is_invalid=0 and language in ('C','C++')
and last_triaged is not null
group by project_id;'''
tr=get(q)

In [84]:
q='''select p.name, lines_of_code
from snapshot s
join
    (select project_id, max(id) as last_snapshot
    from snapshot group by project_id) t
on s.id=t.last_snapshot
join project p on s.project_id = p.id;'''
loc=get(q)

In [101]:
df=reduce(lambda x,y: pd.merge(x,y,on='name'),[ts,dates,intvl,ta,tr,loc])
df = df.sort_values('total_alerts', ascending=False)
df.triaged = round(df.triaged/df.total_alerts *100,2)

In [103]:
df.total_scans=df.total_scans.apply(lambda x: "{:,}".format(x))
df.total_alerts=df.total_alerts.apply(lambda x: "{:,}".format(x))
df.lines_of_code=df.lines_of_code.apply(lambda x: "{:,}".format(x))
df=df.rename(columns={'name':'Project', 'total_scans':'Scan Reports',
            'start_date':'Start Date','end_date':'End Date', 
            'intvl':'Scan Interval (days)', 'total_alerts':'Total                      Alerts', 'triaged':'Triaged Alerts (%)', 'lines_of_code':
            'Lines of Code' })

In [104]:
print(tabulate(df, tablefmt="pipe", headers="keys", showindex=False,        floatfmt=",.2f", numalign="right"))

| Project               | Scan Reports   | Start Date   | End Date   |   Scan Interval (days) | Total                      Alerts   |   Triaged Alerts (%) | Lines of Code   |
|:----------------------|:---------------|:-------------|:-----------|-----------------------:|:------------------------------------|---------------------:|:----------------|
| Linux                 | 648            | 2012-05-17   | 2020-06-15 |                      3 | 19,514                              |                13.66 | 13,959,859      |
| Firefox               | 662            | 2006-02-22   | 2018-10-27 |                      2 | 12,944                              |                36.33 | 8,223,984       |
| LibreOffice           | 414            | 2012-10-13   | 2020-06-26 |                      4 | 11,982                              |                55.42 | 6,130,847       |
| Samba                 | 770            | 2006-02-23   | 2020-06-17 |                      3 | 4,507                        